In [ ]:
using AlfvenDetectors
using GenModels
#using PyPlot
using Plots
using BSON
using Flux
using ValueHistories
using StatsBase
using Random

In [ ]:
using PyCall
umap = pyimport("umap")

In [ ]:
mf = "/home/vit/vyzkum/alfven/experiments/conv/uprobe/ConvWAE_channels-[8,16]_patchsize-128_nepochs-1_2019-05-07T15:09:31.304.bson"
model_data = BSON.load(mf)
exp_args = model_data[:experiment_args]
model_args = model_data[:model_args]
model_kwargs = model_data[:model_kwargs]
history = model_data[:history]
model = Flux.testmode!(GenModels.construct_model(mf));

In [ ]:
# now get some data
datapath = "/home/vit/vyzkum/alfven/cdb_data/uprobe_data"
patchsize = 128
readfun = AlfvenDetectors.readnormlogupsd
shotnos, labels, tstarts, fstarts = AlfvenDetectors.labeled_patches()
patchdata = map(x->AlfvenDetectors.get_patch(datapath,x[1], x[2], x[3], patchsize, readfun;
	memorysafe = true)[1],	zip(shotnos, tstarts, fstarts))
data = cat(patchdata..., dims=4)


In [ ]:
Z = model.encoder(data).data;
gZ = model.pz(400);

In [ ]:
using Plots
gr()

In [ ]:
Plots.scatter(Z[1,:], Z[2,:], Z[3,:])
Plots.scatter!(gZ[1,:], gZ[2,:], gZ[3,:])

In [ ]:
modelpath = "/home/vit/vyzkum/alfven/experiments/conv_old_library/uprobe/"

In [ ]:
datapath = "/home/vit/vyzkum/alfven/cdb_data/uprobe_data/"
labeled_shots, shot_labels = AlfvenDetectors.labeled_data()
Random.seed!(123)
iused = sample(1:size(labeled_shots,1), 10, replace=false)
labeled_shots = labeled_shots[iused]
shot_labels = shot_labels[iused]
datafiles = readdir(datapath);
println("$(length(datafiles)) shots available")

In [ ]:
patchsize = 128
readfun = AlfvenDetectors.readnormlogupsd
data_labels = []
data_shotnos = []
data = Array{Float32,4}(undef,patchsize,patchsize,1,0)
for (label, shot) in zip(shot_labels, labeled_shots)
    file = joinpath.(datapath, datafiles[map(x->occursin("$shot", x),datafiles)])
    X = AlfvenDetectors.collect_conv_signals(file, readfun, patchsize)
    N = size(X,4)
    data = cat(data,X,dims=4)
    push!(data_labels, fill(label,N))
    push!(data_shotnos, fill(shot,N))
end
data_labels = vcat(data_labels...);
data_shotnos = vcat(data_shotnos...);

In [ ]:
testdata = readfun(joinpath(datapath, "uprobe_10893.h5"));
testdata = testdata[129:384, 1537:2816]
figure()
pcolormesh(testdata)
testdata = AlfvenDetectors.split_reshape(testdata,patchsize);
size(testdata)

In [ ]:
mf = joinpath(modelpath, "batchnorm-test/ConvAE_xdim-(128, 128, 1)_ldim-64_nlayers-2_kernelsize-3_channels-[2, 4]_scaling-2_batchnorm-true_batchsize-128_nepochs-1000_opt-RMSProp_eta-0.001_usegpu-true_memoryefficient-true_2019-04-05T17:03:35.079.bson")
# this is the best model, however it was trained on the 10893 data
#mf = joinpath(modelpath, "outbatchnorm/ConvAE_xdim-(128, 128, 1)_ldim-64_nlayers-2_kernelsize-3_channels-[2, 4]_scaling-2_outbatchnorm-false_batchnorm-true_batchsize-128_nepochs-1000_opt-RMSProp_eta-0.001_usegpu-true_memoryefficient-true_2019-04-11T22:20:35.763.bson")
#mf = joinpath(modelpath, "outbatchnorm/ConvAE_xdim-(128, 128, 1)_ldim-64_nlayers-2_kernelsize-3_channels-[2, 4]_scaling-2_outbatchnorm-true_batchnorm-true_batchsize-128_nepochs-1000_opt-RMSProp_eta-0.001_usegpu-true_memoryefficient-true_2019-04-11T20:12:06.821.bson")
model_data = BSON.load(mf)
params = parse_params(mf)
hist = model_data[:history]
model = model_data[:model]
if get(params, :batchnorm, false)
    Flux.testmode!(model);
end
display(params)
is,ls = get(hist,:loss)
vl = AlfvenDetectors.loss(model, testdata).data
println("trainig loss: $(ls[end])")
println("validation loss: $vl")

Check the reconstructions.

In [ ]:
cmap="plasma"
for i in 1:size(testdata,4)
    figure(figsize=(10,5))
    subplot(1,2,1)
    pcolormesh(testdata[:,:,1,i], cmap=cmap)
    subplot(1,2,2)
    pcolormesh(model(testdata[:,:,:,i:i]).data[:,:,1,1], cmap=cmap)
    
end

In [ ]:
testdata

In [ ]:
GC.gc()

In [ ]:
# now compute the code in batches
batchsize = 10
z = model.encoder(data[:,:,:,1:batchsize]).data;
for i in 2:floor(Int,size(data,4)/batchsize)
    z=cat(z, model.encoder(data[:,:,:,((i-1)*batchsize+1):i*batchsize]).data, dims=2)
end
z=cat(z, model.encoder(data[:,:,:,end-(size(data,4)%batchsize)+1:end]).data, dims=2);

In [ ]:
plt.hist(mean(z,dims=2))

In [ ]:
umap_model = umap.UMAP(n_components = 2, n_neighbors=15, min_dist=0.1)

In [ ]:
z2D = Array(umap_model.fit_transform(z')')

In [ ]:
scatter(z2D[1,:],z2D[2,:],s=5)
title("data from shots $(labeled_shots)")

In [ ]:
for shotno in unique(data_shotnos)
    zs = z2D[:,data_shotnos.==shotno]
    scatter(zs[1,:], zs[2,:],s=5,label="$shotno")
end
legend()

In [ ]:
for label in unique(data_labels)
    zs = z2D[:,data_labels.==label]
    scatter(zs[1,:], zs[2,:],s=5,label="$label")
end
legend()

In [ ]:
testz = model.encoder(testdata).data;
testz2D = Array(umap_model.transform(testz')')

In [ ]:
for label in unique(data_labels)
    zs = z2D[:,data_labels.==label]
    scatter(zs[1,:], zs[2,:],s=5,label="$label")
end
scatter(testz2D[1,:], testz2D[2,:],s=5,label="test",c="k")
legend()

In [ ]:
clusterinds = collect(1:size(z2D,2))[ (-50. .< z2D[1,:] .< -10) .&  (-10 .< z2D[2,:] .< 10)]
#clusterinds = collect(1:size(z2D,2))[ (-2. .< z2D[1,:] .< 1) .&  (0 .< z2D[2,:] .< 3)]
#clusterinds = collect(1:size(z2D,2))[ (0.5 .< z2D[1,:] .< 3) .&  (5.5 .< z2D[2,:] .< 6.5)]
scatter(z2D[1,clusterinds],z2D[2,clusterinds])
scatter(testz2D[1,:], testz2D[2,:],s=5,label="test",c="k")


In [ ]:
for i in clusterinds
    figure()
    pcolormesh(data[:,:,1,i], cmap="plasma")
    title("$i, label = $(data_labels[i])")
end

In [ ]:
plt.hist(mean(z,dims=2))

In [ ]:
plt.hist(mean(z[:,clusterinds],dims=2))